# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [3]:
!tar cvfz pipelines.tar.gz*

tar: Cowardly refusing to create an empty archive
Try 'tar --help' or 'tar --usage' for more information.


In [75]:
# import libraries
import pandas as pd
import numpy as np
import sqlalchemy

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier


[nltk_data] Error loading punkt: HTTP Error 502: Gateway Error
[nltk_data] Error loading stopwords: HTTP Error 503: Service
[nltk_data]     Unavailable
[nltk_data] Error loading wordnet: HTTP Error 503: Service Unavailable
[nltk_data] Error loading averaged_perceptron_tagger: HTTP Error 503:
[nltk_data]     Service Unavailable


In [11]:
df.columns

Index(['id', 'message', 'original', 'genre', 'related', 'request', 'offer',
       'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
       'security', 'military', 'child_alone', 'water', 'food', 'shelter',
       'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [37]:
# load data from database
engine = sqlalchemy.create_engine('sqlite:///df.db').connect()
df = pd.read_sql_table('df', engine)
X = df['message'].values
Y = df[['related', 'request', 'offer',
       'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
       'security', 'military', 'child_alone', 'water', 'food', 'shelter',
       'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report']].values

### 2. Write a tokenization function to process your text data

In [36]:
X

array(['Weather update - a cold front from Cuba that could pass over Haiti',
       'Is the Hurricane over or is it not over',
       'Looking for someone but no name', ...,
       "Proshika, operating in Cox's Bazar municipality and 5 other unions, Ramu and Chokoria, assessment, 5 kg rice, 1,5 kg lentils to 700 families.",
       'Some 2,000 women protesting against the conduct of the elections were teargassed as they tried to converge on the local electoral commission offices in the southern oil city of Port Harcourt.',
       'A radical shift in thinking came about as a result of this meeting, recognizing that HIV/AIDS is at the core of the humanitarian crisis and identifying the crisis itself as a function of the HIV/AIDS pandemic.'], dtype=object)

In [28]:
def tokenize(text):
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    lemmatizer = WordNetLemmatizer()
    # lemmatize andremove stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stopwords.words('english')]

    return tokens 

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [64]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged).values

In [31]:
pipeline = Pipeline([
    
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),

        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [66]:
x_train, x_test, y_train, y_test = train_test_split(X, Y,
                                                   train_size=0.8,
                                                   random_state = 42)

pipeline.fit(x_train, y_train)

/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


IndexError: list index out of range

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [39]:
preds = pipeline.predict(x_test)

In [47]:
preds[:, 3]

array(['0', '0', '1', ..., '0', '0', '1'], dtype=object)

In [42]:
from sklearn.metrics import classification_report

In [50]:
for i in range(preds.shape[1]):
    print(f'The {i} column')
    print(classification_report(y_test[:, i], preds[:, i]))
    print('-'*50)

The 0 column
             precision    recall  f1-score   support

          0       0.63      0.47      0.54      1266
          1       0.84      0.91      0.87      3938
          2       0.40      0.42      0.41        40

avg / total       0.79      0.80      0.79      5244

--------------------------------------------------
The 1 column
             precision    recall  f1-score   support

          0       0.89      0.97      0.93      4349
          1       0.77      0.44      0.56       895

avg / total       0.87      0.88      0.87      5244

--------------------------------------------------
The 2 column
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5218
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      5244

--------------------------------------------------
The 3 column
             precision    recall  f1-score   support

          0       0.76      0.86      0.81 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


             precision    recall  f1-score   support

          0       0.95      1.00      0.98      4974
          1       0.72      0.11      0.19       270

avg / total       0.94      0.95      0.93      5244

--------------------------------------------------
The 6 column
             precision    recall  f1-score   support

          0       0.98      1.00      0.99      5117
          1       0.64      0.06      0.10       127

avg / total       0.97      0.98      0.97      5244

--------------------------------------------------
The 7 column
             precision    recall  f1-score   support

          0       0.98      1.00      0.99      5156
          1       0.00      0.00      0.00        88

avg / total       0.97      0.98      0.97      5244

--------------------------------------------------
The 8 column
             precision    recall  f1-score   support

          0       0.97      1.00      0.99      5089
          1       0.60      0.08      0.14       155

av

### 6. Improve your model
Use grid search to find better parameters. 

In [51]:
from sklearn.model_selection import GridSearchCV

In [55]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f5fe1b24950>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [56]:
parameters = {
        'clf__estimator__n_estimators': [50, 100, 200],
        'clf__estimator__min_samples_split': [2, 3, 4]

    }

cv = GridSearchCV(pipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [74]:
cv.fit(x_train, y_train)

KeyboardInterrupt: 

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [76]:
import joblib
joblib.dump(pipeline, 'filename.pkl', compress = 1)

['filename.pkl']

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.